In [17]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset, TensorDataset
from torch.optim import Adam,SGD
import numpy as np

device='cuda' if torch.cuda.is_available() else 'cpu'

In [2]:
X_train = torch.tensor(
                        [[[[1,2,3,4],[2,3,4,5],[5,6,7,8],[1,3,4,5]]],
                        [[[-1,2,3,-4],[2,-3,4,5],[-5,6,-7,8],[-1,-3,-4,-5]]]]
                       ).to(device).float()
X_train /= 8    
y_train = torch.tensor([0,1],dtype=torch.float32).to(device)

In [3]:
def get_model():
    model = nn.Sequential(
        nn.Conv2d(1, 1, kernel_size=3),
        nn.MaxPool2d(2),
        nn.ReLU(),
        nn.Flatten(),
        nn.Linear(1,1),
        nn.Sigmoid()
    ).to(device)
    loss=nn.BCELoss()
    optimizer = Adam(model.parameters(), lr=1e-2)
    return model,loss,optimizer

In [4]:
from torchsummary import summary
model,loss,optimizer = get_model()
summary(model, X_train);

Layer (type:depth-idx)                   Output Shape              Param #
├─Conv2d: 1-1                            [-1, 1, 2, 2]             10
├─MaxPool2d: 1-2                         [-1, 1, 1, 1]             --
├─ReLU: 1-3                              [-1, 1, 1, 1]             --
├─Flatten: 1-4                           [-1, 1]                   --
├─Linear: 1-5                            [-1, 1]                   2
├─Sigmoid: 1-6                           [-1, 1]                   --
Total params: 12
Trainable params: 12
Non-trainable params: 0
Total mult-adds (M): 0.00
Input size (MB): 0.00
Forward/backward pass size (MB): 0.00
Params size (MB): 0.00
Estimated Total Size (MB): 0.00


In [5]:
def train_batch(model, loss, optimizer, X_batch, y_batch):
    model.train()
    optimizer.zero_grad()
    y_pred = model(X_batch)
    loss_value = loss(y_pred.view(-1), y_batch)
    loss_value.backward()
    optimizer.step()
    return loss_value

In [6]:
trn_dl = DataLoader(TensorDataset(X_train, y_train))

In [7]:
for epoch in range(200):
    for X_batch, y_batch in trn_dl:
        loss_value = train_batch(model, loss, optimizer, X_batch, y_batch)

In [8]:
model(X_train[:1])

tensor([[0.1493]], grad_fn=<SigmoidBackward0>)

A Toy example just to showcase how it works, not that its a generalizable implementation

In [9]:
list(model.children())

[Conv2d(1, 1, kernel_size=(3, 3), stride=(1, 1)),
 MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False),
 ReLU(),
 Flatten(start_dim=1, end_dim=-1),
 Linear(in_features=1, out_features=1, bias=True),
 Sigmoid()]

In [10]:
(cnn_w, cnn_b), (lin_w, lin_b) = [(layer.weight.data,layer.bias.data) for layer in list(model.children()) if hasattr(layer, 'weight')]

In [11]:
h_im, w_img = X_train.shape[-2:]
h_cnn, w_cnn = cnn_w.shape[-2:]
sumprod=torch.zeros(h_im-h_cnn+1,w_img-w_cnn+1)

In [12]:
for i in range(h_im-h_cnn+1):
    for j in range(w_img-w_cnn+1):
        sumprod[i,j] = torch.sum(X_train[0,0,i:i+h_cnn,j:j+w_cnn]*cnn_w.reshape(3,3))+cnn_b

In [13]:
pooling_layer_output=torch.max(sumprod)

In [14]:
pooling_layer_output.clamp_min_(0)

tensor(0.)

In [15]:
intermediate_output = pooling_layer_output*lin_w+lin_b

In [16]:
F.sigmoid(intermediate_output)

tensor([[0.1493]])